In [1]:
!pip install pandas pyodbc numpy scikit-image scikit-learn matplotlib Pillow swifter joblib opencv-python-headless varname scipy seaborn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyodbc
from varname import nameof
from scipy.stats import f_oneway
from sklearn.cluster import kmeans_plusplus
from sklearn.cluster import k_means
import seaborn as sns
import warnings
import math
warnings.filterwarnings('ignore')

In [3]:
def get_otsu_threshold(image):
    thresh, _ = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

In [4]:
def snr(image):
    thresh = get_otsu_threshold(image)
    ridge_pixels = image[image >= thresh]
    valley_pixels = image[image < thresh]
    mean_ridge = np.mean(ridge_pixels)
    mean_valley = np.mean(valley_pixels)
    var_ridge = np.var(ridge_pixels)
    var_valley = np.var(valley_pixels)
    snr_value = ((mean_ridge - mean_valley) ** 2 / (var_ridge + var_valley))
    return snr_value

In [5]:
def strength(image):
    thresh = get_otsu_threshold(image)
    ridge_pixels = image[image >= thresh]
    valley_pixels = image[image < thresh]
    mean_ridge = np.mean(ridge_pixels)
    mean_valley = np.mean(valley_pixels)
    return mean_ridge - mean_valley

In [6]:
def continuity(image):
    thresh = get_otsu_threshold(image)
    _, binary = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY_INV)
    num, labels = cv2.connectedComponents(binary)
    num_ridge_pixels = np.count_nonzero(binary)
    return (num - 1) / num_ridge_pixels

In [7]:
def clarity(image):
    thresh = get_otsu_threshold(image)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilated = cv2.morphologyEx(image, cv2.MORPH_DILATE, kernel)
    eroded = cv2.morphologyEx(image, cv2.MORPH_ERODE, kernel)

    ridge_gradient = np.abs(image - eroded)
    valley_gradient = np.abs(image - dilated)

    _, ridge_mask = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)
    _, valley_mask = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY_INV)

    ridge_clarity_map = np.multiply(ridge_mask, ridge_gradient) - np.multiply(valley_mask, valley_gradient)
    clarity = np.sum(ridge_clarity_map) / len(image.ravel())

    return clarity

In [8]:
def get_fp(df):
    for i in range(1, 81):
        for j in range(1, 81):
            if (i-1) // 8 == (j-1) // 8:
                continue
            if float((df[f'{i}'][j]).strip("%")) >= 50:
                yield { i: j } 

In [9]:
def get_fn(df):
    for i in range(1, 81):
        for j in range(1, 81):
            if (i-1) // 8 != (j-1) // 8 or i == j:
                continue
            if float((df[f'{i}'][j]).strip("%")) < 50:
                yield { i: j } 

In [10]:
def get_tp(df):
    for i in range(1, 81):
        for j in range(1, 81):
            if (i-1) // 8 != (j-1) // 8 or i == j:
                continue
            if float((df[f'{i}'][j]).strip("%")) >= 50:
                yield { i: j } 

In [11]:
def get_tn(df):
    for i in range(1, 81):
        for j in range(1, 81):
            if (i-1) // 8 == (j-1) // 8:
                continue
            if float((df[f'{i}'][j]).strip("%")) < 50:
                yield { i: j } 

In [12]:
def get_macro_stats(df):
    class_stats = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
        9: 0,
        10: 0,
    }
    for i in range(0, 10):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for j in range(i*8+1, (i+1)*8+1):
            has_positive_inside = False
            has_positive_outside = False
            for k in range(1, 81):
                score = float((df[f'{j}'][k]).strip("%"))
                if score >= 50:
                    if (k-1) // 8 == i:
                        has_positive_inside = True
                    else:
                        has_positive_outside = True
            if has_positive_inside:
                tp += 1
            else:
                fn += 1
            if has_positive_outside:
                fp += 1
            else:
                tn += 1
        class_stats[i+1] = {
            "TP": tp,
            "FP": fp,
            "TN": tn,
            "FN": fn
        }
    return class_stats
            

In [13]:
def get_macro_stats_new(df, bound):
    class_stats = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
        9: 0,
        10: 0,
    }
    for i in range(0, 10):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for j in range(1, 81):
            score = df[f'group{i+1}'][j]
            if score >= bound:
                if (j-1) // 8 == i:
                    tp += 1
                else:
                    fp += 1
            else:
                if (j-1) // 8 == i:
                    fn += 1
                else:
                    tn += 1
        class_stats[i+1] = {
            "TP": tp,
            "FP": fp,
            "TN": tn,
            "FN": fn
        }
    return class_stats

def get_macro_stats_new2(df, bound1, bound2, shift):
    class_stats = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
        9: 0,
        10: 0,
    }
    for i in range(0, 10):
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for j in range(shift+1, shift+81):
            score1 = df[f'group{i+1}_pos'][j]
            score2 = df[f'group{i+1}_mea'][j]
            if score1 >= bound1 and score2 >= bound2:
                if (j-shift-1) // 8 == i:
                    tp += 1
                else:
                    fp += 1
            else:
                if (j-shift-1) // 8 == i:
                    fn += 1
                else:
                    tn += 1
        class_stats[i+1] = {
            "TP": tp,
            "FP": fp,
            "TN": tn,
            "FN": fn
        }
    return class_stats
            

In [14]:
def get_micro_stats(df, bound):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for j in range(1, 81):
        has_positive_inside = False
        has_positive_outside = False
        for k in range(0, 80):
            score = float((df[f'{j}'][k]).strip("%"))
            if score >= bound:
                if k // 8 == (j-1) // 8:
                    tp += 1
                else:
                    fp += 1
            else:
                if k // 8 == (j-1) // 8:
                    fn += 1
                else:
                    tn += 1
        # if has_positive_inside:
        #     tp += 1
        # else:
        #     fn += 1
        # if has_positive_outside:
        #     fp += 1
        # else:
        #     tn += 1
    return {
        "TP": tp,
        "FP": fp,
        "TN": tn,
        "FN": fn
    }

In [15]:
def get_macro_metrics(class_stats):
    class_metrics = {}
    for class_id in class_stats.keys():
        tp = class_stats[class_id]["TP"]
        fp = class_stats[class_id]["FP"]
        tn = class_stats[class_id]["TN"]
        fn = class_stats[class_id]["FN"]
        acc = (tp+tn)/(tp+fp+tn+fn)*100
        if tp+fp == 0:
            pre = 100
        else:
            pre = tp/(tp+fp)*100
        if tp+fn == 0:
            rec = 0
        else:
            rec = tp/(tp+fn)*100
        spec = tn/(tn+fp)*100
        fpr = fp/(fp+tn)*100
        if pre+rec == 0:
            f1 = 0
            f05 = 0
        else:
            f1 = 2*pre*rec/(pre+rec)
            f05 = 1.25*pre*rec/(0.25*pre+rec)
        class_metrics[class_id] = {
            "accuracy": acc,
            "precision": pre,
            "recall": rec,
            "specifity": spec,
            "FPR": fpr,
            "F1": f1,
            "F0.5": f05
        }
    return class_metrics

In [16]:
def get_macro_average(macro_stats):
    accuracies = np.array([value["accuracy"] for _, value in macro_stats.items()])
    precisions = np.array([value["precision"] for _, value in macro_stats.items()])
    recalls = np.array([value["recall"] for _, value in macro_stats.items()])
    specifities = np.array([value["specifity"] for _, value in macro_stats.items()])
    fprs = np.array([value["FPR"] for _, value in macro_stats.items()])
    f1s = np.array([value["F1"] for _, value in macro_stats.items()]),
    f05s = np.array([value["F0.5"] for _, value in macro_stats.items()]),
    return {
        "accuracy": np.average(accuracies),
        "precision": np.average(precisions),
        "recall": np.average(recalls),
        "specifity": np.average(specifities),
        "FPR": np.average(fprs),
        "F1": np.average(f1s),
        "F0.5": np.average(f05s),
        "AUC ROC": np.average(recalls) / 100 + np.average(specifities) / 100 - np.average(recalls) / 100 * np.average(specifities) / 100,
        "AUC PR": np.average(recalls) / 100 * np.average(precisions) / 100
    }

In [17]:
def visualize(metric_data, title):
    for class_label, values in metric_data.items():
        sns.histplot(data=values, bins=10, kde=True, label=class_label)
    plt.legend(title="Classes")
    plt.title(f"{title} histogram")
    plt.show()

    data_to_plot = [values for values in metric_data.values()]
    mins = [float(min(values)) for values in metric_data.values()]
    maxs = [float(max(values)) for values in metric_data.values()]
    fig, ax = plt.subplots()
    sns.boxplot(data=data_to_plot, ax=ax)
    plt.xticks(ticks=range(0, len(list(metric_data.keys()))), labels=list(metric_data.keys()))
    plt.title(f"{title} box plot")
    ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
    # Show the minor grid as well. Style it in very light gray as a thin,
    # dotted line.
    ax.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
    # Make the minor ticks and gridlines show.
    ax.minorticks_on()
    ax.yaxis.grid(True) # Hide the horizontal gridlines
    ax.xaxis.grid(False) # Show the vertical gridlines
    plt.show()

    sns.violinplot(data=data_to_plot, inner="quartile")
    plt.xticks(ticks=range(0, len(list(metric_data.keys()))), labels=list(metric_data.keys()))
    plt.title(f"{title} box plot")
    plt.show()

In [18]:
def get_micro_metrics(stats):
    tp = stats["TP"]
    fp = stats["FP"]
    tn = stats["TN"]
    fn = stats["FN"]
    acc = (tp+tn)/(tp+fp+tn+fn)*100
    pre = tp/(tp+fp)*100
    rec = tp/(tp+fn)*100
    spec = tn/(tn+fp)*100
    fpr = fp/(fp+tn)*100
    f1 = 2*pre*rec/(pre+rec)
    f05 = 1.25*pre*rec/(0.25*pre+rec)
    mcc = (tp*tn - fp*fn) / (math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    bac = (rec + spec) / 2
    return {
        # "accuracy": acc,
        # "precision": pre,
        # "recall": rec,
        # "specifity": spec,
        "F0.5": f05 / 100,
        "MCC": mcc,
        "FPR": fpr / 100,
        # "F1": f1,
        "BAC": bac / 100,
        # "AUC ROC": (1 + rec - fpr) / 2
    }

In [20]:
df = pd.read_csv("E:\\PHD\\Fingerprint.Refactored\\Demo\\Demo\\bin\\Debug\\net9.0\\results.csv")
stats = get_micro_stats(df, 10)
metrics = get_micro_metrics(stats)
print(metrics)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PHD\\Fingerprint.Refactored\\Demo\\Demo\\bin\\Debug\\net9.0\\results.csv'

In [19]:
df = pd.read_csv("image.csv")
centroid, label, inertia = k_means(
    df[['SNR', 'Ridge strength', 'Ridge continuity', 'Ridge clarity']].to_numpy(), n_clusters=4, n_init="auto", random_state=0
)
pd.DataFrame(label, columns=['Label']).to_csv("test1.csv")


In [30]:
# conn_string = ('Driver={ODBC Driver 18 for SQL Server};SERVER=localhost;'
#                'Database=Fingerprint;TrustServerCertificate=yes;Trusted_Connection=yes;Integrated Security=yes;')
# context = pyodbc.connect(conn_string)
# image_info = pd.read_sql("SELECT * FROM Fingerprint.dbo.Images", context)
# snrs = {
#     "TP": [],
#     "FP": [],
#     "TN": [],
#     "FN": []
# }
# strengths = {
#     "TP": [],
#     "FP": [],
#     "TN": [],
#     "FN": []
# }
# continuities = {
#     "TP": [],
#     "FP": [],
#     "TN": [],
#     "FN": []
# }
# clarities = {
#     "TP": [],
#     "FP": [],
#     "TN": [],
#     "FN": []
# }
shift = 10
df = pd.read_csv("results2_new2.csv")
df = df.set_index('Unnamed: 0')
df1 = df.applymap(lambda x: float(x.strip("%")))
for i in range(0, 10):
    j = (i+shift)*8
    df_tmp = df1[[f'{j+1}', f'{j+2}', f'{j+3}', f'{j+4}', f'{j+5}', f'{j+6}', f'{j+7}', f'{j+8}']]
    count = df_tmp[df_tmp >= 50.00].count(axis=1)
    positives = df_tmp[df_tmp >= 50.00].mean(axis=1).fillna(0)
    mean = df_tmp.mean(axis=1).fillna(0)
    positives1 = pd.merge(positives.to_frame(), count.to_frame(), left_index=True, right_index=True)
    mean1 = pd.merge(mean.to_frame(), count.to_frame(), left_index=True, right_index=True)
    df_test_positive = pd.merge(df1, positives1, left_index=True, right_index=True)
    df_test_mean = pd.merge(df1, mean1, left_index=True, right_index=True)
    df1[f"group{i+1}_pos"] = df_test_positive['0_x']*df_test_positive['0_y']
    df1[f"group{i+1}_mea"] = df_test_mean['0_x']*df_test_mean['0_y']
df2 = df1[['group1_pos', 'group2_pos', 'group3_pos', 'group4_pos', 'group5_pos', 'group6_pos', 'group7_pos', 'group8_pos', 'group9_pos', 'group10_pos', 'group1_mea', 'group2_mea', 'group3_mea', 'group4_mea', 'group5_mea', 'group6_mea', 'group7_mea', 'group8_mea', 'group9_mea', 'group10_mea']] 
df2 = df2.applymap(lambda x: round(x, 2) / 8)
# print(df2)
stats = get_macro_stats_new2(df2, 15, 1, shift*8)
metrics = get_macro_metrics(stats)
avg = get_macro_average(metrics)
print(avg)
# results = {}
# lists=[]
# for b1 in range(1, 100):
#     for b2 in range(1, 100):
#         stats = get_macro_stats_new2(df2, b1, b2, shift*8)
#         metrics = get_macro_metrics(stats)
#         avg = get_macro_average(metrics)
#         results[(b1, b2)] = avg
#         # if (avg['FPR'] == 0.0):
#         #     results[(b1, b2)] = avg
#         #     continue
# best_f1 = 0
# best_key = None
# best_value = None
# for key, value in results.items():
#     lists.append(value['F1'])
#     if value['F1'] > best_f1:
#         best_f1 = value['F1']
#         best_key = key
#         best_value = value
# print(best_key)
# print(best_value)
# plt.plot(range(len(lists)), lists)
# plt.show()
# df2.to_csv("mean.csv")
# for i in range(0, 10):
#     avg = (df1[f'{i*8+1}'] + df1[f'{i*8+2}'] + df1[f'{i*8+3}'] + df1[f'{i*8+4}'] + df1[f'{i*8+5}'] + df1[f'{i*8+6}'] + df1[f'{i*8+7}'] + df1[f'{i*8+8}']) / 8
#     df1[f"group{i+1}"] = (abs(df1[f'{i*8+1}'] - avg) + abs(df1[f'{i*8+2}'] - avg) + abs(df1[f'{i*8+3}'] - avg) + abs(df1[f'{i*8+4}'] - avg) + abs(df1[f'{i*8+5}'] - avg) + abs(df1[f'{i*8+6}'] - avg) + abs(df1[f'{i*8+7}'] - avg) + abs(df1[f'{i*8+8}'] - avg)) / 8
# df2 = df1[['group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7', 'group8', 'group9', 'group10']] 
# df2 = df2.applymap(lambda x: round(x, 2))
# df2.to_csv("mad.csv")
    # print("------------------------------------------METRICS (macro-approach)-------------------------------------------------------")
# macro_stats = get_macro_stats(df)
# macro_metrics = get_macro_metrics(macro_stats)
# for i in range(1, 11):
#     print(f"Class {i} metrics")
#     print(macro_metrics[i])
# print("Averaged")
# print(get_macro_average(macro_metrics))
# print("------------------------------------------METRICS (micro-approach)-------------------------------------------------------")
# micro_stats = get_micro_stats(df)
# micro_metrics = get_micro_metrics(micro_stats)
# print(micro_metrics)
# for i in range(1, 81):
#     for j in range(1, 81):
#         if i == j:
#             df[f"{i}"][j] = "100.00%"
#         else:
#             score = float((df[f'{i}'][j]).strip("%"))
#             row1 = image_info[image_info.Id == i].iloc[0]
#             row2 = image_info[image_info.Id == j].iloc[0]
#             image1 = cv2.imread(row1.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
#             image1 = image1[row1.HeightShift:row1.HeightOffset, row1.WidthShift:row1.WidthOffset]
#             image2 = cv2.imread(row2.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
#             image2 = image2[row2.HeightShift:row2.HeightOffset, row2.WidthShift:row2.WidthOffset]
#             snr_diff = abs(snr(image1) - snr(image2))
#             strength_diff = abs(strength(image1) - strength(image2))
#             cont_diff = abs(continuity(image1) - continuity(image2))
#             clar_diff = abs(clarity(image1) - clarity(image2))
#             if score >= 50:
#                 if clar_diff > 80:
#                     df[f"{i}"][j] = "0.00%"
# print(len(list(get_fn(df))))
# print(len(list(get_fp(df))))
# print(len(list(get_tn(df))))
# print(len(list(get_tp(df))))
# # print("------------------------------FALSE NEGATIVES------------------------------------")
# # for pair in list(get_fn(df)):
# #     for key in pair.keys():
# #         row1 = image_info[image_info.Id == key].iloc[0]
# #         row2 = image_info[image_info.Id == pair[key]].iloc[0]
# #         image1 = cv2.imread(row1.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image1 = image1[row1.HeightShift:row1.HeightOffset, row1.WidthShift:row1.WidthOffset]
# #         image2 = cv2.imread(row2.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image2 = image2[row2.HeightShift:row2.HeightOffset, row2.WidthShift:row2.WidthOffset]
# #         snrs["FN"].append(abs(snr(image1) - snr(image2)))
# #         strengths["FN"].append(abs(strength(image1) - strength(image2)))
# #         continuities["FN"].append(abs(continuity(image1) - continuity(image2)))
# #         clarities["FN"].append(abs(clarity(image1) - clarity(image2)))
# # print("------------------------------FALSE POSITIVES------------------------------------")
# # for pair in list(get_fp(df)):
# #     for key in pair.keys():
# #         row1 = image_info[image_info.Id == key].iloc[0]
# #         row2 = image_info[image_info.Id == pair[key]].iloc[0]
# #         image1 = cv2.imread(row1.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image1 = image1[row1.HeightShift:row1.HeightOffset, row1.WidthShift:row1.WidthOffset]
# #         image2 = cv2.imread(row2.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image2 = image2[row2.HeightShift:row2.HeightOffset, row2.WidthShift:row2.WidthOffset]
# #         snrs["FP"].append(abs(snr(image1) - snr(image2)))
# #         strengths["FP"].append(abs(strength(image1) - strength(image2)))
# #         continuities["FP"].append(abs(continuity(image1) - continuity(image2)))
# #         clarities["FP"].append(abs(clarity(image1) - clarity(image2)))
# # print("------------------------------TRUE NEGATIVES------------------------------------")
# # for pair in list(get_tn(df)):
# #     for key in pair.keys():
# #         row1 = image_info[image_info.Id == key].iloc[0]
# #         row2 = image_info[image_info.Id == pair[key]].iloc[0]
# #         image1 = cv2.imread(row1.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image1 = image1[row1.HeightShift:row1.HeightOffset, row1.WidthShift:row1.WidthOffset]
# #         image2 = cv2.imread(row2.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image2 = image2[row2.HeightShift:row2.HeightOffset, row2.WidthShift:row2.WidthOffset]
# #         snrs["TN"].append(abs(snr(image1) - snr(image2)))
# #         strengths["TN"].append(abs(strength(image1) - strength(image2)))
# #         continuities["TN"].append(abs(continuity(image1) - continuity(image2)))
# #         clarities["TN"].append(abs(clarity(image1) - clarity(image2)))
# # print("------------------------------TRUE POSITIVES------------------------------------")
# test = pd.DataFrame(columns=['name', 'SNR', 'Ridge strength', 'Ridge continuity', 'Ridge clarity'])
# for i in range(101, 111):
#     for j in range(1, 9):
#         image = cv2.imread(f'{i}_{j}.png', cv2.IMREAD_GRAYSCALE)
#         test.loc[(i-101)*8+j] = [f'{i}_{j}.png', round(snr(image), 2), round(strength(image), 2), round(continuity(image)*1000, 2), round(clarity(image), 2)]
# test.to_csv('image.csv')       
# # for pair in list(get_tp(df)):
# #     for key in pair.keys():
# #         row1 = image_info[image_info.Id == key].iloc[0]
# #         row2 = image_info[image_info.Id == pair[key]].iloc[0]
# #         image1 = cv2.imread(row1.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image1 = image1[row1.HeightShift:row1.HeightOffset, row1.WidthShift:row1.WidthOffset]
# #         image2 = cv2.imread(row2.FileName.replace(".tif", ".png"), cv2.IMREAD_GRAYSCALE)
# #         image2 = image2[row2.HeightShift:row2.HeightOffset, row2.WidthShift:row2.WidthOffset]
# #         snrs["TP"].append(abs(snr(image1) - snr(image2)))
# #         strengths["TP"].append(abs(strength(image1) - strength(image2)))
# #         continuities["TP"].append(abs(continuity(image1) - continuity(image2)))
# #         clarities["TP"].append(abs(clarity(image1) - clarity(image2)))
# # snr_data = {
# #     "TP": snrs["TP"],
# #     "TN": snrs["TN"],
# #     "FP": snrs["FP"],
# #     "FN": snrs["FN"]
# # }
# # rs_data = {
# #     "TP": strengths["TP"],
# #     "TN": strengths["TN"],
# #     "FP": strengths["FP"],
# #     "FN": strengths["FN"]
# # }
# # rc_data = {
# #     "TP": continuities["TP"],
# #     "TN": continuities["TN"],
# #     "FP": continuities["FP"],
# #     "FN": continuities["FN"]
# # }
# # rcl_data = {
# #     "TP": clarities["TP"],
# #     "TN": clarities["TN"],
# #     "FP": clarities["FP"],
# #     "FN": clarities["FN"]
# # }
# # visualize(snr_data, "SNR")
# # visualize(rs_data, "Ridge strength")
# # visualize(rc_data, "Ridge continuity")
# # visualize(rcl_data, "Ridge clarity")
# # snr_data = {
# #     "Positive": snrs["TP"] + snrs["TN"],
# #     "Negative": snrs["FP"] + snrs["FN"]
# # }
# # rs_data = {
# #     "Positive": strengths["TP"] + strengths["TN"],
# #     "Negative": strengths["FP"] + strengths["FN"]
# # }
# # rc_data = {
# #     "Positive": continuities["TP"] + continuities["TN"],
# #     "Negative": continuities["FP"] + continuities["FN"]
# # }
# # rcl_data = {
# #     "Positive": clarities["TP"] + clarities["TN"],
# #     "Negative": clarities["FP"] + clarities["FN"]
# # }
# # visualize(snr_data, "SNR")
# # visualize(rs_data, "Ridge strength")
# # visualize(rc_data, "Ridge continuity")
# # visualize(rcl_data, "Ridge clarity")
# # snr_anova = f_oneway(snrs["TP"] + snrs["TN"], snrs["FP"] + snrs["FN"])
# # strength_anova = f_oneway(strengths["TP"] + strengths["TN"], strengths["FP"] + strengths["FN"])
# # continuity_anova = f_oneway(continuities["TP"] + continuities["TN"], continuities["FP"] + continuities["FN"])
# # clarity_anova = f_oneway(clarities["TP"] + clarities["TN"], clarities["FP"] + clarities["FN"])
# # print(snr_anova)
# # print(strength_anova)
# # print(continuity_anova)
# # print(clarity_anova)
# # for class_name in ["TP", "TN", "FP", "FN"]:
# #     print(class_name)
# #     for metric in [snrs, strengths, continuities, clarities]:
# #         print(f"Mean = {np.mean(metric[class_name]):.2f}")
# #         print(f"Median = {np.median(metric[class_name]):.2f}")
# #         print(f"Standard deviation = {np.std(metric[class_name]):.2f}")
# #         print(f"Variance = {np.var(metric[class_name]):.2f}")
# #         print(f"25th, 50th, 75th percentile = {np.percentile(metric[class_name], [25, 50, 75])}")
# #         print("--------------------------------------------------------------------------------")

{'accuracy': 98.25, 'precision': 98.88888888888889, 'recall': 83.75, 'specifity': 99.86111111111111, 'FPR': 0.1388888888888889, 'F1': 88.74509803921569, 'F0.5': 93.81313131313131, 'AUC ROC': 0.9997743055555557, 'AUC PR': 0.8281944444444445}
